In [15]:
!pip install awswrangler --q

Keyring is skipped due to an exception: 'keyring.backends'


In [16]:
import os
import io
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import datetime

import awswrangler as wr
#import dask.dataframe as dd

import warnings
warnings.filterwarnings("ignore")

#### eciw household 

In [17]:
path = "/root/Lapse/data/"

In [18]:
filenames = glob.glob(path + "*household*.csv")
print(filenames)

['/root/Lapse/data/vul_eciw_household_monthly_2022_4.csv', '/root/Lapse/data/vul_eciw_household_monthly_2022_2.csv', '/root/Lapse/data/vul_eciw_household_monthly_2022_6.csv', '/root/Lapse/data/vul_eciw_household_monthly_2022_1.csv', '/root/Lapse/data/vul_eciw_household_monthly_2022_5.csv', '/root/Lapse/data/vul_eciw_household_monthly_2022_3.csv']


In [19]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    d = pd.read_csv(f, encoding="ISO-8859-1") 
    dfs.append(d) 
eciw_hh = pd.concat(dfs, ignore_index=True)
del dfs
print(eciw_hh.shape)

/root/Lapse/data/vul_eciw_household_monthly_2022_4.csv
/root/Lapse/data/vul_eciw_household_monthly_2022_2.csv
/root/Lapse/data/vul_eciw_household_monthly_2022_6.csv
/root/Lapse/data/vul_eciw_household_monthly_2022_1.csv
/root/Lapse/data/vul_eciw_household_monthly_2022_5.csv
/root/Lapse/data/vul_eciw_household_monthly_2022_3.csv
(4618147, 192)


In [20]:
eciw_hh['as_of_hist_dt']=eciw_hh['as_of_hist_dt'].astype('datetime64[ns]')
eciw_hh['hhld_acq_dt']=eciw_hh['hhld_acq_dt'].astype('datetime64[ns]')
eciw_hh['most_rcnt_pur_dt']=eciw_hh['most_rcnt_pur_dt'].astype('datetime64[ns]')

In [21]:
eciw_hh = eciw_hh[['household_id', 
               #dates
               'as_of_hist_dt', 'hhld_acq_dt', 'most_rcnt_pur_dt',
               # codes
               'incm_rng_cd', 'prim_occup_cd','prmr_prsnc_chld_ind', 'net_wrth_gold_cd',
             'hhld_qdrnt_cd',
               #asset mix
              'wc_tot_asset_amt','wc_asset_mix_anty_amt','wc_asset_mix_bond_amt',
               'wc_asset_mix_dep_amt','wc_asset_mix_mf_amt','wc_asset_mix_oth_asset_amt',
               'wc_asset_mix_stk_amt']]

#### eciw subject 

In [22]:
filenames = glob.glob(path + "*subject*.csv")
print(filenames)

['/root/Lapse/data/vul_eciw_subject_monthly_2022_3.csv', '/root/Lapse/data/vul_eciw_subject_monthly_2022_5.csv', '/root/Lapse/data/vul_eciw_subject_monthly_2022_1.csv', '/root/Lapse/data/vul_eciw_subject_monthly_2022_4.csv', '/root/Lapse/data/vul_eciw_subject_monthly_2022_6.csv', '/root/Lapse/data/vul_eciw_subject_monthly_2022_2.csv']


In [23]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    d = pd.read_csv(f, encoding="ISO-8859-1") 
    dfs.append(d) 
eciw_sb = pd.concat(dfs, ignore_index=True)
del dfs
print(eciw_sb.shape)

/root/Lapse/data/vul_eciw_subject_monthly_2022_3.csv
/root/Lapse/data/vul_eciw_subject_monthly_2022_5.csv
/root/Lapse/data/vul_eciw_subject_monthly_2022_1.csv
/root/Lapse/data/vul_eciw_subject_monthly_2022_4.csv
/root/Lapse/data/vul_eciw_subject_monthly_2022_6.csv
/root/Lapse/data/vul_eciw_subject_monthly_2022_2.csv
(4753268, 58)


In [24]:
eciw_sb['as_of_hist_dt']=eciw_sb['as_of_hist_dt'].astype('datetime64[ns]')
eciw_sb['subj_acq_dt']=eciw_sb['subj_acq_dt'].astype('datetime64[ns]')

In [25]:
eciw_sb = eciw_sb[['household_id', 'subject_id', 'agmt_pkge_id','policy_id',
               'as_of_hist_dt', 'subj_acq_dt',
               'mrtl_stat_cd', 'gndr_cd', 'subj_state_cd', 'subj_act_ind',
               'pref_lang_cd','deceased_ind', 'rtl_dstrb_orphn_cd',
               'ixi_buoyancy', 'ixi_fin_cohort_cd', 
               'subj_aum_amt' ]]

#### join 

In [26]:
df =eciw_sb.merge(eciw_hh, how='left', 
                left_on=['household_id', 'as_of_hist_dt'], 
                right_on = ['household_id', 'as_of_hist_dt'],
                suffixes=('', '_remove'))
df.drop([i for i in df.columns if 'remove' in i],axis=1, inplace=True)

In [ ]:
df = df.sort_values(['household_id', 'as_of_hist_dt'])

In [27]:
df = df.drop_duplicates()

In [28]:
print(df.shape)

(4753268, 30)


#### export

In [29]:
%%time
import awswrangler as wr

wr.s3.to_parquet(
    df=df,
    path='s3://cdo-ililapse-364524684987-bucket/x266754/lapse/eciw_jan_jun.parquet'
    #dataset=True
)

CPU times: user 20.1 s, sys: 6.43 s, total: 26.5 s
Wall time: 16.1 s


{'paths': ['s3://cdo-ililapse-364524684987-bucket/x266754/lapse/eciw_jan_jun.parquet'],
 'partitions_values': {}}